In [1]:
!pip install duckdb --no-index --find-links=/kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
!pip install python-gdcm
!pip install pylibjpeg
!pip install pylibjpeg-libjpeg==2.2.0
!pip install pylibjpeg-openjpeg==2.3.0
!pip install matplotlib==3.10.3
!pip install scikit-learn==1.7.0
!pip install polars --no-index --find-links=/kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
!pip install pydicom

Looking in links: /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg/duckdb-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 23.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 5.9 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.9 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available:

In [2]:
from pydicom import dcmread
from pydicom.dataset import FileDataset, FileMetaDataset
from pydicom.uid import generate_uid, ImplicitVRLittleEndian

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
import duckdb as dd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
import pickle
import gc
import ctypes
from pathlib import Path
import logging
import json
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor
import datetime
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow import keras

E0000 00:00:1757180023.245371      10 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:230


In [3]:
print(tf.__version__)
print(tfio.__version__)

2.18.0
0.37.1


In [4]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

print("Number of accelerators: ", tpu_strategy.num_replicas_in_sync)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1757180231.152611      10 service.cc:148] XLA service 0x5d53527a9b50 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1757180231.152662      10 service.cc:156]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1757180231.152666      10 service.cc:156]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1757180231.152669      10 service.cc:156]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1757180231.152673      10 service.cc:156]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1757180231.152676      10 service.cc:156]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1757180231.152678      10 service.cc:156]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1757180231.152681      10 service.cc:156]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1757180231.152684      10 service.cc:156]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [9]:
print("All devices: ", tf.config.list_logical_devices('TPU'))

All devices:  [LogicalDevice(name='/device:TPU:0', device_type='TPU'), LogicalDevice(name='/device:TPU:1', device_type='TPU'), LogicalDevice(name='/device:TPU:2', device_type='TPU'), LogicalDevice(name='/device:TPU:3', device_type='TPU'), LogicalDevice(name='/device:TPU:4', device_type='TPU'), LogicalDevice(name='/device:TPU:5', device_type='TPU'), LogicalDevice(name='/device:TPU:6', device_type='TPU'), LogicalDevice(name='/device:TPU:7', device_type='TPU')]


In [5]:
pl.Config(fmt_str_lengths=100)
pl.Config.set_tbl_rows(100)

polars.config.Config

In [6]:
allowed_tags = ['BitsAllocated', 'BitsStored', 'Rows', 'Columns', 'FrameOfReferenceUID', 'HighBit', 'ImageOrientationPatient'
                , 'ImagePositionPatient', 'InstanceNumber', 'Modality', 'PhotometricInterpretation'
                , 'PixelRepresentation', 'PixelSpacing', 'PlanarConfiguration', 'RescaleIntercept', 'RescaleSlope'
                , 'RescaleType', 'SamplesPerPixel', 'SliceThickness', 'SpacingBetweenSlices']

In [7]:
df_all_data = pl.read_parquet('/kaggle/input/rsna-aneurysm-train-metadata-suman/full_training_data.parquet')
print("Shape of training metadata", df_all_data.shape)
df_all_data.columns

Shape of training metadata (1001346, 24)


['file_name',
 'image_shape',
 'coordinates_x',
 'coordinates_y',
 'coordinates_f',
 'aneurysm_present_in_series',
 'aneurysm_present_in_image',
 'seriesinstanceuid',
 'patientage',
 'patientsex',
 'modality',
 'left_infraclinoid_internal_carotid_artery',
 'right_infraclinoid_internal_carotid_artery',
 'left_supraclinoid_internal_carotid_artery',
 'right_supraclinoid_internal_carotid_artery',
 'left_middle_cerebral_artery',
 'right_middle_cerebral_artery',
 'anterior_communicating_artery',
 'left_anterior_cerebral_artery',
 'right_anterior_cerebral_artery',
 'left_posterior_communicating_artery',
 'right_posterior_communicating_artery',
 'basilar_tip',
 'other_posterior_circulation']

In [10]:
with tf.device('/device:TPU:0'):
    root_folder = "/kaggle/input/rsna-intracranial-aneurysm-detection/series"
    
    def create_full_image_path(row_data):
        return row_data['root_folder'] + '/' + row_data['seriesinstanceuid'] + '/' + row_data['file_name']
    
    df_all_data = df_all_data.with_columns(pl.lit(root_folder).alias("root_folder"))
    
    df_all_data = df_all_data.with_columns([
        pl.struct(pl.col("root_folder"), pl.col("seriesinstanceuid"), pl.col("file_name"))
        .map_elements(create_full_image_path, return_dtype=pl.String)
        .alias("full_image_path")
    ])

In [12]:
def extract_single_frame(multiframe_path, slice_number, output_path=None):
    """
    Extract a single frame from a multi-frame DICOM
    
    Args:
        multiframe_path: Path to multi-frame DICOM file
        slice_number: The slice number to extract (0-based index)
        output_path: Path to save the single-frame DICOM. If None, returns the dataset
    """
    try:
        # Read the multi-frame DICOM with force=True to handle potentially corrupted files
        multi_ds = dcmread(multiframe_path, force=True)
        
        # Verify it's a multi-frame image
        if not hasattr(multi_ds, 'NumberOfFrames'):
            raise ValueError("Input DICOM is not a multi-frame image")
        
        # Check if slice number is valid
        if slice_number >= multi_ds.NumberOfFrames:
            raise ValueError(f"Slice number {slice_number} is out of range. "
                           f"Image has {multi_ds.NumberOfFrames} frames")
        
        # Create new dataset for single frame
        single_ds = FileDataset(output_path or "temp.dcm", {}, 
                              file_meta=FileMetaDataset(), 
                              preamble=b"\0" * 128)
        
        # Copy attributes from multi-frame dataset
        attrs_to_copy = allowed_tags
        
        for attr in attrs_to_copy:
            if hasattr(multi_ds, attr):
                setattr(single_ds, attr, getattr(multi_ds, attr))
        
        # Generate new UIDs
        single_ds.SOPInstanceUID = generate_uid()
        single_ds.file_meta.MediaStorageSOPInstanceUID = single_ds.SOPInstanceUID
        
        # Set transfer syntax to uncompressed little endian
        single_ds.file_meta.TransferSyntaxUID = ImplicitVRLittleEndian
        single_ds.file_meta.MediaStorageSOPClassUID = multi_ds.file_meta.MediaStorageSOPClassUID
        if hasattr(multi_ds.file_meta, 'ImplementationClassUID'):
            single_ds.file_meta.ImplementationClassUID = multi_ds.file_meta.ImplementationClassUID
        
        # Set instance-specific attributes
        single_ds.InstanceNumber = slice_number + 1
        
        try:
            # Try to get pixel array directly
            pixel_array = multi_ds.pixel_array[slice_number]
        except Exception as e:
            #print(f"Warning: Could not directly access pixel_array: {e}")
            # Alternative approach: decompress and get pixels
            if hasattr(multi_ds, 'decompress'):
                multi_ds.decompress()
            pixel_array = multi_ds.pixel_array[slice_number]
        
        # Set pixel data
        single_ds.PixelData = pixel_array.tobytes()
        
        # Update image-specific attributes
        single_ds.NumberOfFrames = 1
        
        # Try to copy position and orientation
        try:
            if hasattr(multi_ds, 'PerFrameFunctionalGroupsSequence'):
                frame_content = multi_ds.PerFrameFunctionalGroupsSequence[slice_number]
                
                if hasattr(frame_content, 'PlanePositionSequence'):
                    position = frame_content.PlanePositionSequence[0].ImagePositionPatient
                    single_ds.ImagePositionPatient = position
                
                if hasattr(frame_content, 'PlaneOrientationSequence'):
                    orientation = frame_content.PlaneOrientationSequence[0].ImageOrientationPatient
                    single_ds.ImageOrientationPatient = orientation
        except Exception as e:
            #print(f"Warning: Could not copy position/orientation: {e}")
            raise
        
        # Add creation timestamp
        dt = datetime.datetime.now()
        single_ds.ContentDate = dt.strftime('%Y%m%d')
        single_ds.ContentTime = dt.strftime('%H%M%S.%f')
        
        # Save or return the dataset
        if output_path:
            single_ds.save_as(output_path, write_like_original=False)
            return None
        return single_ds
    
    except Exception as e:
        #print(f"Error extracting frame: {e}")
        raise

# Alternative version using different approach for compressed files
def extract_single_frame_alternative(multiframe_path, slice_number, output_path=None):
    """
    Alternative version for handling problematic files
    """
    try:
        # Read with force and stop before pixels
        multi_ds = dcmread(multiframe_path, force=True, stop_before_pixels=True)
        
        # Read pixel data separately
        with open(multiframe_path, 'rb') as f:
            multi_ds.PixelData = f.read()
        
        # Decompress if needed
        if hasattr(multi_ds, 'decompress'):
            multi_ds.decompress()
        
        # Get pixel array
        pixel_array = multi_ds.pixel_array[slice_number]
        
        # Create new dataset
        single_ds = FileDataset(output_path or "temp.dcm", {}, 
                              file_meta=FileMetaDataset(), 
                              preamble=b"\0" * 128)
        
        # Copy attributes (same as before)
        attrs_to_copy = allowed_tags
        
        for attr in attrs_to_copy:
            if hasattr(multi_ds, attr):
                setattr(single_ds, attr, getattr(multi_ds, attr))
        
        # Generate new UIDs
        single_ds.SOPInstanceUID = generate_uid()
        single_ds.file_meta.MediaStorageSOPInstanceUID = single_ds.SOPInstanceUID
        
        # Set transfer syntax to uncompressed little endian
        single_ds.file_meta.TransferSyntaxUID = ImplicitVRLittleEndian
        single_ds.file_meta.MediaStorageSOPClassUID = multi_ds.file_meta.MediaStorageSOPClassUID
        if hasattr(multi_ds.file_meta, 'ImplementationClassUID'):
            single_ds.file_meta.ImplementationClassUID = multi_ds.file_meta.ImplementationClassUID
        
        # Set instance-specific attributes
        single_ds.InstanceNumber = slice_number + 1
        
        try:
            # Try to get pixel array directly
            pixel_array = multi_ds.pixel_array[slice_number]
        except Exception as e:
            #print(f"Warning: Could not directly access pixel_array: {e}")
            # Alternative approach: decompress and get pixels
            if hasattr(multi_ds, 'decompress'):
                multi_ds.decompress()
            pixel_array = multi_ds.pixel_array[slice_number]
        
        # Set pixel data
        single_ds.PixelData = pixel_array.tobytes()
        
        # Update image-specific attributes
        single_ds.NumberOfFrames = 1
        
        # Try to copy position and orientation
        try:
            if hasattr(multi_ds, 'PerFrameFunctionalGroupsSequence'):
                frame_content = multi_ds.PerFrameFunctionalGroupsSequence[slice_number]
                
                if hasattr(frame_content, 'PlanePositionSequence'):
                    position = frame_content.PlanePositionSequence[0].ImagePositionPatient
                    single_ds.ImagePositionPatient = position
                
                if hasattr(frame_content, 'PlaneOrientationSequence'):
                    orientation = frame_content.PlaneOrientationSequence[0].ImageOrientationPatient
                    single_ds.ImageOrientationPatient = orientation
        except Exception as e:
            #print(f"Warning: Could not copy position/orientation: {e}")
            raise
        
        # Add creation timestamp
        dt = datetime.datetime.now()
        single_ds.ContentDate = dt.strftime('%Y%m%d')
        single_ds.ContentTime = dt.strftime('%H%M%S.%f')
        
        # Save or return the dataset
        if output_path:
            single_ds.save_as(output_path, write_like_original=False)
            return None
        return single_ds
        
    except Exception as e:
        #print(f"Error in alternative extraction: {e}")
        raise

# Function to try both methods
def safe_extract_single_frame(multiframe_path, slice_number, output_path=None):
    """
    Try both extraction methods
    """
    try:
        return extract_single_frame(multiframe_path, slice_number, output_path)
    except Exception as e:
        #print(f"Primary method failed: {e}")
        #print("Trying alternative method...")
        try:
            return extract_single_frame_alternative(multiframe_path, slice_number, output_path)
        except Exception as e2:
            #print(f"Alternative method also failed: {e2}")
            raise

# Version with zoom functionality
def load_and_view_single_slice_with_zoom(dcm_path, x_coord, y_coord, f_coord=None, zoom_size=100):
    """
    Load and display a single DICOM slice with crosshair and zoomed inset
    
    Args:
        dcm_path: Path to the DICOM file
        x_coord: x coordinate for the crosshair
        y_coord: y coordinate for the crosshair
        zoom_size: Size of the zoom window in pixels
    """
    # Read DICOM file
    if f_coord:
        ds = safe_extract_single_frame(dcm_path, f_coord)
    else:
        ds = dcmread(dcm_path)
    img = ds.pixel_array
    
    # Create figure and axes
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
    
    # Main image with crosshair
    ax1.imshow(img, cmap='gray')
    ax1.axvline(x=x_coord, color='red', alpha=0.5)
    ax1.axhline(y=y_coord, color='red', alpha=0.5)
    ax1.plot(x_coord, y_coord, 'r+', markersize=10, markeredgewidth=2)
    
    # Zoomed region
    x_start = int(max(0, x_coord - zoom_size/2))
    x_end = int(min(img.shape[1], x_coord + zoom_size/2))
    y_start = int(max(0, y_coord - zoom_size/2))
    y_end = int(min(img.shape[0], y_coord + zoom_size/2))
    
    zoomed = img[y_start:y_end, x_start:x_end]
    ax2.imshow(zoomed, cmap='gray')
    
    # Add crosshair to zoomed region
    center_x = x_coord - x_start
    center_y = y_coord - y_start
    ax2.axvline(x=center_x, color='red', alpha=0.5)
    ax2.axhline(y=center_y, color='red', alpha=0.5)
    ax2.plot(center_x, center_y, 'r+', markersize=10, markeredgewidth=2)
    
    ax1.axis('off')
    ax2.axis('off')
    ax1.set_title('Full Image')
    ax2.set_title('Zoomed Region')
    
    plt.tight_layout()
    plt.show()

In [66]:
def read_and_parse_dicom_files_tensorflow_train(dcm_path, f_coord=None):
    if f_coord:
        ds = safe_extract_single_frame(dcm_path, f_coord)
    else:
        ds = dcmread(dcm_path)
    
    arr = ds.pixel_array.astype(np.float32)
    
    # Apply rescale if present
    slope = float(getattr(ds, "RescaleSlope", 1) or 1)
    intercept = float(getattr(ds, "RescaleIntercept", 0) or 0)
    arr = arr * slope + intercept
    
    # Handle MONOCHROME1 (invert)
    if getattr(ds, "PhotometricInterpretation", "") == "MONOCHROME1":
        arr = arr.max() - arr
    
    image = tf.convert_to_tensor(arr)
    
    expanded_image = tf.expand_dims(image, -1)
    m, M=tf.math.reduce_min(expanded_image), tf.math.reduce_max(expanded_image)
    expanded_image = (tf.image.grayscale_to_rgb(expanded_image)-m)/(M-m)
    expanded_image = tf.image.resize(expanded_image, (128,128))
    sqzd_image = tf.squeeze(expanded_image)

    return sqzd_image

def load_dicom_tf(filepath_tensor, f_coord_tensor=None):
    """
    Wrapper function to call _load_dicom_image within a tf.data.Dataset pipeline.
    """
    if f_coord_tensor:
        image = tf.py_function(
            read_and_parse_dicom_files_tensorflow_train,
            inp=[filepath_tensor, f_coord_tensor],
            Tout=tf.float32 # Adjust Tout based on the expected output type of your image data
        )
    else:
        image = tf.py_function(
            read_and_parse_dicom_files_tensorflow_train,
            inp=[filepath_tensor],
            Tout=tf.float32 # Adjust Tout based on the expected output type of your image data
        )
    # Ensure the shape is defined if known, as tf.py_function can lose shape info
    image.set_shape([128, 128]) # Example for a 2D image, adjust as needed
    return image 

def preprocessing(dcm_path, f_coord):
    train_img = load_dicom_tf(dcm_path, f_coord)
    train_img = tf.reshape(train_img, shape=(128, 128, 3))
    return train_img

def load_dataset_tensorflow_train(dcm_path, f_coord, labels):
    image = preprocessing(dcm_path, f_coord)
    return {"images": tf.cast(image, tf.float32), "labels": tf.cast(labels, tf.float32)}

def dict_to_tuple(inputs):
    return inputs["images"], inputs["labels"]

In [49]:
cols_to_select = ['full_image_path', 'coordinates_f', 'left_infraclinoid_internal_carotid_artery',  'right_infraclinoid_internal_carotid_artery',  'left_supraclinoid_internal_carotid_artery'
                  ,  'right_supraclinoid_internal_carotid_artery',  'left_middle_cerebral_artery',  'right_middle_cerebral_artery',  'anterior_communicating_artery'
                  ,  'left_anterior_cerebral_artery',  'right_anterior_cerebral_artery',  'left_posterior_communicating_artery',  'right_posterior_communicating_artery'
                  ,  'basilar_tip',  'other_posterior_circulation'
]

df_for_train_baseline = df_all_data.select(cols_to_select)

In [50]:
x_train, x_test_val = train_test_split(df_for_train_baseline, test_size=0.4, random_state=42)
x_test, x_valid = train_test_split(x_test_val, test_size=0.2, random_state=42)

print("Training data shape : {0}".format(x_train.shape))
print("Test data shape : {0}".format(x_test.shape))
print("Validation data shape : {0}".format(x_valid.shape))

Training data shape : (600807, 15)
Test data shape : (320431, 15)
Validation data shape : (80108, 15)


In [16]:
label_cols = ['left_infraclinoid_internal_carotid_artery',  'right_infraclinoid_internal_carotid_artery',  'left_supraclinoid_internal_carotid_artery'
                  ,  'right_supraclinoid_internal_carotid_artery',  'left_middle_cerebral_artery',  'right_middle_cerebral_artery',  'anterior_communicating_artery'
                  ,  'left_anterior_cerebral_artery',  'right_anterior_cerebral_artery',  'left_posterior_communicating_artery',  'right_posterior_communicating_artery'
                  ,  'basilar_tip',  'other_posterior_circulation'
]

In [22]:
dd.sql("select left_infraclinoid_internal_carotid_artery, count(distinct(full_image_path)) \
from x_train group by left_infraclinoid_internal_carotid_artery").pl()

left_infraclinoid_internal_carotid_artery,count(DISTINCT full_image_path)
i64,i64
0,600721
1,68


In [27]:
with tf.device('/device:TPU:1'):
    x_train_agg = x_train.group_by('left_infraclinoid_internal_carotid_artery').agg(pl.col("full_image_path").count())
    list_of_classes = pl.Series(x_train_agg.select('left_infraclinoid_internal_carotid_artery')).to_list()
    bias_arr_prep_dict = {}
    for cls in list_of_classes:
        print(cls)
        print(x_train_agg.filter(pl.col('left_infraclinoid_internal_carotid_artery')==cls)\
              .select(pl.col('full_image_path')).item(0, 0))
        bias_arr_prep_dict[cls] = x_train_agg.filter(pl.col('left_infraclinoid_internal_carotid_artery')==cls)\
        .select(pl.col('full_image_path')).item(0, 0)
        
    print(bias_arr_prep_dict)

0
600736
1
71
{0: 600736, 1: 71}


In [67]:
def generate_tf_datasets(p_df, p_BATCH_SIZE_PER_REPLICA):

    BATCH_SIZE = p_BATCH_SIZE_PER_REPLICA * tpu_strategy.num_replicas_in_sync
    
    image_filenames = pl.Series(p_df.select(pl.col('full_image_path'))).to_list()
    image_f_coords = pl.Series(p_df.select(pl.col('coordinates_f'))).to_list()
    #image_labels = pl.Series(p_df.select(label_cols)).to_list()
    image_labels = pl.Series(p_df.select(label_cols)).map_elements(lambda d: list(d.values())).to_list()
    
    image_dataset = tf.data.Dataset.from_tensor_slices((image_filenames, image_f_coords, image_labels))
    
    image_ds = image_dataset.map(load_dataset_tensorflow_train, num_parallel_calls=tf.data.AUTOTUNE)
    image_ds = image_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
    image_ds = image_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)
    image_ds = image_ds.prefetch(tf.data.AUTOTUNE)
    
    return image_ds

In [51]:
x_trial = x_train.slice(offset=1, length=3)
x_trial.shape

(3, 15)

In [52]:
x_trial

full_image_path,coordinates_f,left_infraclinoid_internal_carotid_artery,right_infraclinoid_internal_carotid_artery,left_supraclinoid_internal_carotid_artery,right_supraclinoid_internal_carotid_artery,left_middle_cerebral_artery,right_middle_cerebral_artery,anterior_communicating_artery,left_anterior_cerebral_artery,right_anterior_cerebral_artery,left_posterior_communicating_artery,right_posterior_communicating_artery,basilar_tip,other_posterior_circulation
str,i32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.996394934697752…",null,0,0,0,0,0,0,0,0,0,0,0,0,0
"""/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.717658785569984…",null,0,0,0,0,0,0,0,0,0,0,0,0,0
"""/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.928612098243936…",null,0,0,0,0,0,0,0,0,0,0,0,0,0


In [45]:
pl.Series(x_trial.select(label_cols)).map_elements(lambda d: list(d.values())).to_list()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [68]:
with tpu_strategy.scope():
    train_ds = generate_tf_datasets(p_df=x_trial, p_BATCH_SIZE_PER_REPLICA = 10)

ValueError: in user code:

    File "/tmp/ipykernel_10/3137455592.py", line 54, in load_dataset_tensorflow_train  *
        image = preprocessing(dcm_path, f_coord)
    File "/tmp/ipykernel_10/3137455592.py", line 50, in preprocessing  *
        train_img = tf.reshape(train_img, shape=(128, 128, 3))

    ValueError: Cannot reshape a tensor with 16384 elements to shape [128,128,3] (49152 elements) for '{{node Reshape}} = Reshape[T=DT_FLOAT, Tshape=DT_INT32](EagerPyFunc, Reshape/shape)' with input shapes: [128,128], [3] and with input tensors computed as partial shapes: input[1] = [128,128,3].
